In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import re

## Get Teams

In [11]:
url = "https://www.balldontlie.io/api/v1/teams"

r=requests.get(url)
r.status_code

200

In [12]:
data = r.json()
teams_df = pd.DataFrame(data['data'])
teams_df.to_csv("teams") # to view easily

## Get Players

In [28]:
players_url = "https://www.balldontlie.io/api/v1/players"
dfs_list = []
def get_page(number):
    param = {"page" : number, "per_page" : 100}
    r=requests.get(players_url, params = param)
    # if the status is successful, we want to gather and clean the data
    if r.status_code == 200:
        # gather data
        data = r.json()
        players_df = pd.DataFrame(data['data'])
        # clean data
        players_df['team_id'] = players_df['team'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
        players_df['team_abr'] = players_df['team'].apply(lambda x: x.get('abbreviation') if isinstance(x, dict) else None)
        players_df['city'] = players_df['team'].apply(lambda x: x.get('city') if isinstance(x, dict) else None)
        players_df = players_df.drop('team', axis=1)
        # now we want to save to a list
        dfs_list.append(players_df)
    # if the status gives an error, we would like to know what the error is
    else: 
        return print(f'Status code error: {r.status_code}')

In [29]:
# create all dfs using the function defined above
for i in range(0, 51):
    get_page(i)

In [33]:
players_df = pd.concat(dfs_list, ignore_index=True)
players_df.to_csv("players") # to be able to read it easily haha